In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install datasets
! pip install -U accelerate
! pip install -U transformers

## **依照<G>去做分割的Read**

In [ ]:
import pandas as pd

def Read_Cut_G( filename ) :

    with open( f"{ filename }.txt", 'r', encoding='utf-8' ) as f :
        data = f.readlines()


    new_data = {}
    label = []
    g_times = 0
    text = ""

    for i in data :

        if "<G>" in i :

            context = i.split( "<G>" )
            i = context[1]

            g_times = g_times + 1
            if g_times == 2 :
                label = []
                g_times = 0
                text = ""
        elif "<USER>" in i :

            context = i.split( "<USER>" )
            i = context[1]

            if text == "" :
                text =  i.strip()
            else :
                text = text + "\n" + i.strip()
        elif "<LABEL>" in i :

            context = i.split( "<LABEL>" )
            i = context[1]

            label.append( i.strip() )
            text = text.strip()
            new_data[text] = label

            label = []
        elif "<BOT>" in i :

            context = i.split( "<BOT>" )
            i = context[1]


            if text == "" :
                text =  i.strip()
            else :
                text = text + "\n" + i.strip()



    # print(new_data)

    return new_data

## **依照label去做分割的Read**

In [ ]:
import pandas as pd

def Read_Cut_Label( filename ) :

    with open( f"{ filename }.txt", 'r', encoding='utf-8' ) as f :
        data = f.readlines()


    new_data = {}
    label = []
    g_times = 0
    text = ""

    for i in data :

        if "<G>" in i :

            context = i.split( "<G>" )
            i = context[1]

            g_times = g_times + 1
            if g_times == 2 :
                g_times = 0
        elif "<USER>" in i :

            context = i.split( "<USER>" )
            i = context[1]

            if text == "" :
                text =  i.strip()
            else :
                text = text + "<SEP>" + i.strip()


        elif "<LABEL>" in i :

            context = i.split( "<LABEL>" )
            i = context[1]

            label.append( i.strip() )

            text = text.strip()
            new_data[text] = label
            label = []
            text = ""

        elif "<BOT>" in i :

            context = i.split( "<BOT>" )
            i = context[1]


            if text == "" :
                text =  i.strip()
            else :
                text = text + "<SEP>" + i.strip()



    # print(new_data)

    return new_data

二分類才要用

In [ ]:
#用來決定哪些是正確的那些是錯誤的label
def  Input_Label( data, correct_string ) :

    copy_data = data.copy()

    for i in copy_data :
        # print( i )
        if copy_data[ i ][ 0 ] != correct_string:

            copy_data[ i ] = [f'不是{ correct_string }']



    # print( copy_data )
    return copy_data


In [ ]:
#為了將類別進行公平分類(label只有1個的時候)
def Fair_Split_Data( data ) :

  label = []
  all_data = {}
  for i in data :
    if data[i] not in label :
      label.append( data[i] )
      all_data[ data[i][0] ] = []
      all_data[ data[i][0] ].append( i )
    else :
      all_data[ data[i][0] ].append( i )

  return all_data


In [ ]:
#改變資料的型態 從dict變成 text, label 的兩個list
def Change_Fomat_Data( data ) :
    all_data = {}
    all_datas = {}
    texts = []
    labels = []
    for text in data :
        texts.append( text )
        labels.append( data[ text ] )

    all_data[ 'text' ] = texts
    all_data[ 'label' ] = labels

    # all_datas[ name ] = all_data

    return all_data


In [ ]:
#改變資料的型態 從dict變成 text, label 的兩個list
def Change_Fomat_Data_Fair( data ) :
    all_data = {}

    for label in data:
      texts = []
      labels = []
      for text in data[label]:
          texts.append(text)
          labels.append([label])

      all_data[f'{label}_text'] = texts
      all_data[f'{label}_label'] = labels


    return all_data

In [ ]:
def Counter_Label( data ):

    count_dict = {}
    for value in data.values() :
        if value[0] not in count_dict :
            count_dict[ value[0] ] = 1
        else :
            count_dict[ value[0] ] = count_dict[ value[0] ] + 1

    print( count_dict )

Counter_Label( data )

In [ ]:
from datasets import Dataset, DatasetDict, load_dataset
from datasets import concatenate_datasets

filenames = [ '詢問', '訂位', '推薦', '導航', '功能對話' ]

all_data = {}
data = {}
ds_dict = DatasetDict()

time = 0
# print( title )
for filename in filenames :

        # read = Read( filename )

        # print(read)

        # read = Input_Label( read,  "詢問問題" )

        # data.update( read )

        # data_function = Change_Fomat_Data( data )

        # dataset = Dataset.from_dict( data_function )

          read = Read_Cut_Label( filename )

          # print( read )

          data.update( read )

all_data = Fair_Split_Data( data )

print( all_data )

dataset = Change_Fomat_Data_Fair( all_data )

print( dataset )

data_function = {}
data_function['text'] = dataset['功能_text']
data_function['label'] = dataset['功能_label']

ds_dict['function'] = Dataset.from_dict( data_function )

data_function = {}
data_function['text'] = dataset['收集資訊_text']
data_function['label'] = dataset['收集資訊_label']

ds_dict['collect'] = Dataset.from_dict( data_function )

print( ds_dict )


In [ ]:
def Split( ds_dict, train_size, classification ) :

    train_test =  ds_dict[ classification ].train_test_split( train_size=train_size, shuffle=True )


    print( train_test )

    return train_test


train_test1 = Split( ds_dict, 0.9 ,'function' )
train_test2 = Split( ds_dict, 0.8 ,'collect' )


In [ ]:
df1 = pd.DataFrame(train_test1['train'])

df2 = pd.DataFrame(train_test2['train'])

df = pd.concat([df1, df2], axis=0, ignore_index=True )  # 合併


from sklearn.preprocessing import MultiLabelBinarizer

classes = ['收集資訊']
mlb = MultiLabelBinarizer(classes=classes)


labels = mlb.fit_transform(df['label']).astype( "float32" )  # 把label轉化成數字 對應 classes 有:1 無: 0   mlb.fit_transform() 格示[[a],[b]....]

df_final = pd.concat( [df['text'], pd.DataFrame(labels,columns=list(mlb.classes_))],axis=1 ) # 將文字 和 pandas( label ) 組合


text = df_final['text'].tolist()  # 取出文字 變成list

print( text )

In [ ]:
df_final

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification ,DistilBertForTokenClassification, DistilBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")



In [ ]:
import torch
from torch.utils.data import Dataset

class C_Dataset( Dataset ) :
  def __init__( self, label, text ) :
    self.label = label
    self.text = text

  def __len__( self ):
    return len( self.text )

  def __getitem__( self, idx ) :
    text = str( self.text[idx] )
    label = torch.tensor( self.label[idx] )

    encoding = tokenizer( text,  max_length=200, truncation=True, padding="max_length", return_tensors='pt' )

    # print( encoding )

    return {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': label
    }

In [ ]:
train_dataset = C_Dataset( labels, text )

## **原本的模型**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification ,DistilBertForTokenClassification, DistilBertForSequenceClassification
print('labels_len:', len(labels[0]) )
tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")
model = AutoModelForSequenceClassification.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student", num_labels=len(labels[0]), ignore_mismatched_sizes=True)

## **自己的模型**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification ,DistilBertForTokenClassification, DistilBertForSequenceClassification
print('labels_len:', len(labels[0]) )
tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/result/Is_it_ask_restaurant_question/checkpoint-2000", num_labels=len(labels[0]), ignore_mismatched_sizes=True)

In [ ]:
from transformers import  Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = '/content/drive/MyDrive/result/Is_Collect_or_Function',
    num_train_epochs = 300,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 32,
    warmup_steps = 500,
    learning_rate=5e-5,
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps = 10,
    save_steps=1000,
    max_steps=3000,
    run_name='my_experiment_1',
    report_to='none'
    # evaluation_strategy = 'epoch',
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    # data_collator = data_collator,
    train_dataset=train_dataset,
    # compute_metrics=compute_metrics,
    # eval_dataset=test_dataset
)

In [ ]:
trainer.train()

## **TEST**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification ,DistilBertForTokenClassification, DistilBertForSequenceClassification
import torch
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/result/Is_Collect_or_Function/checkpoint-3000")
tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")


In [ ]:



correct_string = "收集資訊"

classes = [ correct_string ]
mlb = MultiLabelBinarizer(classes=classes)

df1 = pd.DataFrame(train_test1['test'])

df2 = pd.DataFrame(train_test2['test'])

df_test = pd.concat([df1, df2], axis=0, ignore_index=True )  # 合併


labels = mlb.fit_transform(df_test['label'])  # 把label轉化成數字 對應 classes 有:1 無: 0   mlb.fit_transform() 格示[[a],[b]....]

labels = labels.tolist()


text = df_test['text'].tolist()


def Predict( text, label ) :

  encoding = tokenizer( text, return_tensors='pt')
  encoding.to( model.device )

  outputs = model( **encoding )


  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid( outputs.logits[0].cpu() )

  print( probs )

  print( text )

  if probs > 0.504 :
    if label == 1 :
      print(f"正確判定({correct_string})")
      return 1
    else:
      print(f"誤判為{correct_string}")
      return 0
  else:
    if label == 0 :
      print("正確判定(other)")
      return 1
    else:
      print(f"{correct_string}誤判為other")
      return 0



def Test_answer( text ) :


  correct = 0

  for la, i in enumerate( text ):

    predict = Predict( i, labels[la][0] )

    if predict == 1 :
      correct = correct + 1

  print(correct)
  print(correct/len(text))



Test_answer( text )


In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix


# 假设你已经加载了你的模型和tokenizer
# model = ...  # 加载模型
# tokenizer = ...  # 加载tokenizer

# 正确的标签字符串
correct_string = "詢問問題"
classes = [correct_string]
mlb = MultiLabelBinarizer(classes=classes)

# 假设 train_test['test'] 是你的测试数据
df_test = pd.DataFrame(train_test['test'])

df_test.to_csv('df_test.csv', index=False)  # 不保存行索引

# 将标签转化为数字
labels = mlb.fit_transform(df_test['label'])  # 对应有:1 无: 0
labels = labels.tolist()
text = df_test['text'].tolist()

def Predict(text):
    # 对输入文本进行编码
    encoding = tokenizer(text, return_tensors='pt')
    encoding = {key: val.to(model.device) for key, val in encoding.items()}  # 确保在正确的设备上

    # 获取模型输出
    outputs = model(**encoding)

    # 应用Sigmoid函数
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(outputs.logits[0].cpu()).detach().numpy()  # 转为NumPy数组以便比较
    return probs

def Test_answer(text):
    correct = 0
    y_true = []
    y_pred = []

    for la, i in enumerate(text):
        # 真实标签
        true_label = labels[la][0]
        y_true.append(true_label)

        # 获取预测概率
        probs = Predict(i)

        # 设置阈值
        threshold = 0.6  # 这里设置你想要的阈值
        predicted_label = (probs >= threshold).astype(int)

        y_pred.append(predicted_label[0])  # 取第一个元素作为预测结果

        # 进行判定
        if predicted_label[0] == 1:
            if true_label == 1:
                print(f"正確判定({correct_string})")
                correct += 1
            else:
                print(f"誤判為{correct_string}")
        else:
            if true_label == 0:
                print("正確判定(other)")
                correct += 1
            else:
                print(f"{correct_string}誤判為other")

    # 计算混淆矩阵
    cm = confusion_matrix(y_true, y_pred)
    print(f"Confusion Matrix:\n{cm}")
    print(f"Total Correct Predictions: {correct}")
    print(f"Accuracy: {correct / len(text):.2f}")

# 调用测试函数
Test_answer(text)
